In [1]:
import pandas as pd
import numpy as np
import os 

os.chdir('../')

In [3]:
df = pd.read_csv('eval_data/bertscorev2.csv')
#df.drop('Unnamed: 0',axis=1,inplace=True)

In [4]:
df.head()

,claude,mixtral,ll70b,ll8b,phi
0,0.492868,0.274986,0.438979,0.434475,0.580536
1,0.531139,0.331079,0.506058,0.311858,0.666477
2,0.467435,0.313752,0.446872,0.466672,0.440016
3,0.456518,0.312949,0.383624,0.359249,0.374173
4,0.534137,0.491245,0.427240,0.445629,0.582361


In [5]:
datasets = ['aqa']*120+['squad']*120+['medqa']*120+['hotpotqa']*120+['triviaqa']*120
df['dataset'] = datasets

In [6]:
df.head()

,claude,mixtral,ll70b,ll8b,phi,dataset
0,0.492868,0.274986,0.438979,0.434475,0.580536,aqa
1,0.531139,0.331079,0.506058,0.311858,0.666477,aqa
2,0.467435,0.313752,0.446872,0.466672,0.440016,aqa
3,0.456518,0.312949,0.383624,0.359249,0.374173,aqa
4,0.534137,0.491245,0.427240,0.445629,0.582361,aqa


In [7]:
subdatasets = ['aqa','squad','medqa','hotpotqa','triviaqa']
models = ['claude','mixtral','ll70b','ll8b','phi']
df.columns = models+['dataset']
dfs =[]
for i in range(len(models)):
    x = df[[models[i],'dataset']]
    x.columns = ['eval','dataset']
    x['model'] = [models[i]]*len(x)
    dfs.append(x)

/tmp/ipykernel_12583/1846329046.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x['model'] = [models[i]]*len(x)
/tmp/ipykernel_12583/1846329046.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x['model'] = [models[i]]*len(x)
/tmp/ipykernel_12583/1846329046.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide

In [8]:
dfs[0].head()

,eval,dataset,model
0,0.492868,aqa,claude
1,0.531139,aqa,claude
2,0.467435,aqa,claude
3,0.456518,aqa,claude
4,0.534137,aqa,claude


In [9]:
df1 = pd.concat(dfs, axis=0, ignore_index=True )

In [10]:
df1.head()

,eval,dataset,model
0,0.492868,aqa,claude
1,0.531139,aqa,claude
2,0.467435,aqa,claude
3,0.456518,aqa,claude
4,0.534137,aqa,claude


In [11]:
len(df1[df1['dataset']=='aqa'])

600

In [12]:
df1.to_csv('eval_data/bertscorev3.csv',index=False)

In [13]:
df = pd.read_csv('export/combinedv3.csv')

In [14]:
df.head()

,Unnamed: 0,questions,answers,dataset,prediction,eval,model
0,0,Where is the Hoppings funfair held?,Town Moor,aqa,"According to the passage, the Hoppings funfair...",1,claude
1,1,Who shares a name with an older type of transp...,Stagecoach North East,aqa,"Based on the information provided, the company...",1,claude
2,2,What would be done for people who need more in...,link the local networks to national networks,aqa,"Based on the given context, there isn't specif...",0,claude
3,3,What was affected by the refurbishment?,cinema,aqa,"Based on the given context, the Pilgrim Street...",1,claude
4,4,"Of the successful state schools listed, this s...",Heaton Manor School,aqa,"To answer this question, I'll list the success...",1,claude


In [15]:
from sklearn.metrics import matthews_corrcoef
r = []
for i in range(len(subdatasets)):
    sub1 = df[df['dataset']==subdatasets[i]]['eval']
    sub2 = df1[df1['dataset']==subdatasets[i]]['eval']
    #sub2 = sub2.fillna(-1)
    result = matthews_corrcoef(np.array(sub1),np.where(np.array(sub2) > 0.5, 1, 0))
    r.append(result)
    
r

[0.1822586359025753,
 0.07781661889626787,
 0.03322888801774506,
 0.299316779661239,
 0.10818818790843408]

In [16]:
r = []
for i in range(len(models)):
    sub1 = df[df['model']==models[i]]['eval']
    sub2 = df1[df1['model']==models[i]]['eval']
    sub2 = sub2.fillna(0)
    result = matthews_corrcoef(np.array(sub1),np.where(np.array(sub2) > 0.5, 1, 0))
    r.append(result)
    
r

[0.24094010661409593,
 0.18154509608800368,
 0.29236096055589056,
 0.35211273551212685,
 0.3428402946814411]

In [44]:
len(df1[df1['dataset']==subdatasets[i]]['eval'])

600

In [48]:
df1['eval'].isna().sum()

15